In [ ]:
# System modules
import pickle
import folium
from folium import plugins
import numpy as np
import pandas as pd
from time import time
from scipy.stats import norm
from datetime import datetime
import matplotlib.pyplot as plt

# User modules
from domain.base import DataRequest
from domain.base import Station
import domain.file_io as file_io
import domain.mongodb_engine as mongodb
from domain.elevation_service import (
    ElevationServiceConnector
)

from helpers.utils import (
    get_station_coordinates,
    add_station_elevations,
    select_near,
    add_alias
)
import helpers.knmi_obs_ingest as knmi
%matplotlib inline

In [ ]:
def plot_markers_on_map(station_dict):
    map_1 = folium.Map(
        [51.3, 5.6],
        zoom_start=10, 
#         tiles='stamentoner'
    )

    for station_id in station_dict:
        station = station_dict[station_id]
        folium.Marker(
            [station.latitude, station.longitude], 
            popup="Station id: %s" % station.station_id,
            icon=folium.Icon(color=station.color)
        ).add_to(map_1)
    return map_1

In [ ]:
# Read station meta data
with open('data/stations.csv', 'r') as f:
    station_meta_data = pd.read_csv(f)
print("Loaded meta data for %d stations." % len(station_meta_data))

In [ ]:
# Defining a request for the Netherlands
request = DataRequest()
start_dt = datetime(2016, 4, 1, 0, 0)
end_dt = datetime(2016, 5, 1, 0, 0)
request.start_datetime = start_dt
request.end_datetime = end_dt
request.time_resolution = 10
request.region = (52.000, 4.790, 51.880, 5.080)  # Cabauw area
# request.region = (53.680, 2.865, 50.740, 7.323)  # The Netherlands
print("Request defined from %s to %s" % (start_dt, end_dt))

In [ ]:
# Defining a request for the Netherlands
from domain.base import DataRequest
request = DataRequest()
start_dt = datetime(2016, 4, 1, 0, 0)
end_dt = datetime(2016, 5, 1, 0, 0)
request.start_datetime = start_dt
request.end_datetime = end_dt
request.time_resolution = 10
request.region = (53.680, 2.865, 50.740, 7.323)
print("Request defined from %s to %s" % (start_dt, end_dt))

knmi_path = '/Volumes/various/netatmo/obs/'
knmi_fsengine = knmi.FileSystemEngine(knmi_path)
obs_data = knmi_fsengine.query(request)

In [ ]:
# Load KNMI 10-minute observations from disk.
print("Querying KNMI observation data..")
knmi_path = '/Volumes/various/netatmo/obs/'
knmi_fsengine = knmi.FileSystemEngine(knmi_path)
obs_data = knmi_fsengine.query(request)

In [ ]:
# Dump observation data to a single file for caching.
with open('data/netherlands_obs_3-25_4-15.pkl', 'wb') as f:
    pickle.dump(obs_data, f)

In [ ]:
# # Load cached observations.
# load_start = time()
# with open('data/netherlands_obs_3-25_4-15.pkl', 'rb') as f:
#     obs_data = pickle.load(f)
# print("Done loading observations (%fs)." % (time() - load_start))

In [ ]:
# # TODO Remove me on new pickled obs file
# obs_data.sort_values('valid_datetime', axis=0, inplace=True)

In [ ]:
# Query netatmo data from disk.
netatmo_path = '/Volumes/various/netatmo/data/'
print("Querying NetAtmo file system engine..")
response = file_io.query(netatmo_path, request)
data_map = response.data_map
print("%d stations loaded." % len(data_map))

In [ ]:
db_connector = mongodb.MongoDBConnector()

In [ ]:
stations = list(db_connector.db.stations.find(
    {},
    {'_id': 0}
))
data_map = {s['station_id']: Station.fromdict(s) for s in stations}

In [ ]:
stations_with_rain = {}
for station_id in data_map:
    station = data_map[station_id]
    if station.hydro_module is not None:
        stations_with_rain[station_id] = station

In [ ]:
# Insert one station
station_ids = list(data_map.keys())
sample_station = data_map[station_ids[1]]
pprint(sample_station.__dict__)

In [ ]:
# 
query = {'_id': 'baz'}
# TODO Only do $push if a measurement is present.
update = {
    '$setOnInsert': {
        '_id': 'baz',
        'elevation': sample_station.elevation,
        'latitude': sample_station.latitude,
        'longitude': sample_station.longitude,
        'station_id': sample_station.station_id,
        'thermo_module': sample_station.thermo_module,
        'hydro_module': sample_station.hydro_module
    },
    '$push': {
            'thermo_module.humidity': {'$each': sample_station.thermo_module['humidity']},
            'thermo_module.pressure': {'$each': sample_station.thermo_module['pressure']},
            'thermo_module.temperature': {'$each': sample_station.thermo_module['temperature']},
            'thermo_module.valid_datetime': {'$each': sample_station.thermo_module['valid_datetime']}
    }}
result = db_connector.db.stations.update_one(query, update, True)

In [ ]:
# Query one station
from pprint import pprint
# returned_sample_station = db_connector.db.stations.find_one({'_id': station_ids[1]})
returned_sample_station = db_connector.db.stations.find_one({'_id': 'baz'})
pprint(returned_sample_station)

In [ ]:
# Find all stations
pprint(list(db_connector.db.stations.find()))

In [ ]:
# Insert all stations
for station_id in data_map:
    db_connector.insert_stations(station_ids)

In [ ]:
cursor = db_connector.db.stations.find({'station_id': {'$in': station_ids}})

In [ ]:
station_id = station_ids[1]
# pprint(data_map[station_id].__dict__)
results = db_connector.db.stations.update_one({'station_id': '1337'}, {'$push':{'thermo_module.humidity': 79, 'thermo_module.pressure': 1017.3, 'temperature': 7.8, 'valid_datetime': datetime(2016, 3, 27, 3, 56, 6)}})

In [ ]:
# Dump NetAtmo data to a single file for caching.
# with open('data/netherlands_3-25_4-15.pkl', 'wb') as f:
with open('data/precip_experiment3.pkl', 'wb') as f:
    pickle.dump(data_map, f)

In [ ]:
# Load cached NetAtmo data.
load_start = time()
# with open("data/netherlands_3-25_4-15.pkl", "rb") as f:
with open("data/precip_experiment3.pkl", "rb") as f:
    data_map = pickle.load(f)
print("Done loading NetAtmo data (%fs)." % (time() - load_start))

In [ ]:
from bson import json_util
import json

with open("data/netatmo_station_data.json", "w") as f:
    for station_id in data_map:
        station = data_map[station_id]    
        json.dump(
            station.__dict__, f, 
            default=json_util.default,
            sort_keys=True
        )

In [ ]:
hydro_stats = []
for station_id in data_map:
    if data_map[station_id].hydro_module is not None:
        hydro_stats.append(station_id)

In [ ]:
thermo_stats = []
for station_id in data_map:
    if data_map[station_id].thermo_module is not None:
        thermo_stats.append(station_id)

In [ ]:
df = pd.DataFrame(data_map[hydro_stats[50]].hydro_module)
df_hourly = df[['hourly_rain_sum', 'time_hour_rain']]
df_hourly.set_index('time_hour_rain').plot()
df_hourly.set_index('time_hour_rain').cumsum().plot()
df_daily = df[['daily_rain_sum', 'time_day_rain']]
df_daily.plot()
df_day_to_hour = df_daily.diff()
df_day_to_hour[df_day_to_hour.daily_rain_sum < 0] = 0
df_day_to_hour.plot()

In [ ]:
# Select stations at or near a specified station id
query_station_id = 6260  # WMO station to search nearby
radius = 5000  # Search radius in meters

# Do search and interpolation
query_station_location = station_meta_data.loc[station_meta_data.stationId == query_station_id, ["latitude", "longitude"]]
print("Query station location: lat %f, lon %f" % (query_station_location.latitude, query_station_location.longitude))

near_station_map = select_near(
                                data_map, 
                                query_station_location.latitude, query_station_location.longitude,
                                radius=radius
                              )

print("%d stations near WMO station %d." % (len(near_station_map), query_station_id))
netatmo.resample_and_interpolate(near_station_map, request.time_resolution)
print("Done.")

In [ ]:
# Construct a map plot of the query station and its found nearby NetAtmo stations.
def plot_local_map(query_station_location, near_station_map): 
    query_lat = list(query_station_location.latitude.values)
    query_lon = list(query_station_location.longitude.values)
    map_1 = folium.Map(
        query_lat + query_lon,
        zoom_start=13, 
        tiles='stamentoner'
    )
    # Draw point of interest
    folium.Marker(query_lat + query_lon, icon=folium.Icon(color='red')).add_to(map_1)

    # Draw search radius
    folium.CircleMarker(
        query_lat + query_lon,
        radius=radius,
        color='#3186cc',
        fill_color='#C7DDF0',
       ).add_to(map_1)

    for station_id in near_station_map:
        station = near_station_map[station_id]
        folium.Marker([station.latitude, station.longitude], popup="Station id: N%d" % station.alias).add_to(map_1)
    map_1
    
plot_local_map(query_station_location, near_station_map)

In [ ]:
# Plotting of timeseries of nearby NetAtmo stations.
fig, (ax1, ax2) = plt.subplots(2, figsize=(200, 10), sharex=True)
ax1.set_xlabel("Valid datetime")

element = 'temperature'
show_cloudiness = False
show_radiation = False
show_grass = False
show_merged = False

show_individual = True

# Construct observation series
gp = obs_data.groupby('station_id')
wmo_obs = gp.get_group(query_station_id)
if element == 'pressure':
    knmi_name = 'p0'
elif element == 'temperature':
    knmi_name = 'ta'
elif element == 'humidity':
    knmi_name = 'rh'
elif element == 'cloudiness':
    knmi_name = 'n'
wmo_obs_el = wmo_obs[wmo_obs.element_name==knmi_name]

print("Station %d has %d observations for element %s" % (query_station_id, len(wmo_obs_el), element))

# == Data processing ==
merged_data = None  # Container for mean data
for station_id in near_station_map:
    station = near_station_map[station_id].thermo_module
    station_alias = 'Station %d' % near_station_map[station_id].alias
    if element in station:
        if merged_data is None:
            merged_data = station[[element]].add_suffix("_N%d" % near_station_map[station_id].alias)
        else:
            merged_data = merged_data.join(station[[element]].add_suffix("_N%d" % near_station_map[station_id].alias),
                                           how='outer')

# # Compute features for quality check
# val_range = (merged_data.max(axis=0) - merged_data.min(axis=0))
# val_sd = merged_data.std(axis=0)
# val_mean = merged_data.mean(axis=0)

# # Merge features
# station_features = pd.concat([val_range, val_sd, val_mean], axis=1)
# station_features.columns = ['range', 'sd', 'mean']

# # The 'is_inside' check currently only works for temperature.
# if element =='temperature':
#     station_features['is_inside'] = np.logical_and(station_features.sd < 2, station_features.range < 10)
# else:
#     station_features['is_inside'] = False

# # Select good stations
# print("%d stations are installed inside and are discarded." % station_features.is_inside.sum())
# outside_station_names = list(station_features[~station_features.is_inside].index)
# merged_data = merged_data[outside_station_names]

# # == Two-tailed Z-test on every date ==
# alpha = 0.04
# # Calculate mean per row
# mean_per_date = merged_data.mean(axis=1)
# std_per_date = merged_data.std(axis=1)
# # Calculate CDF per row
# merged_cdf_vals = merged_data.apply(
#     norm.cdf,
#     axis=0,
#     loc=mean_per_date,
#     scale=std_per_date
# )
# # Filter out values above threshold
# merged_data = merged_data.mask(
#     merged_cdf_vals.applymap(lambda x: (x > 1 - (0.5 * alpha)) or (x < (0.5 * alpha)))
# )

# == Compute root mean squared error ==
# obs_copy = wmo_obs_el.copy()
# obs_copy.set_index('valid_datetime', inplace=True)
# wmo_netatmo = merged_data.mean(axis=1).to_frame(name='netatmo_' + element).join(obs_copy, how='outer')
# wmo_netatmo.rename(
#     columns={
#         'value': 'wmo_' + element
#     },
#     inplace=True
# )
# wmo_netatmo.drop(['element_name', 'station_id'], axis=1, inplace=True)
# error = wmo_netatmo['netatmo_' + element] - wmo_netatmo['wmo_' + element]
# mae = error.abs().mean()
# rmse = np.sqrt(error.apply(np.square).mean())
# print("MAE Merged NetAtmo vs. WMO: %f" % mae)
# print("RMSE Merged NetAtmo vs. WMO: %f" % rmse)

# == First subplot == 
if show_individual:
    for name in merged_data:    
        p = ax1.plot(merged_data.index.values, merged_data[name].values, marker='x', label=name)
            
# Plot mean temperature of stations
# if show_merged:
#     ax1.plot(merged_data.index.values, merged_data.mean(axis=1), label='NetAtmo Mean', linewidth=2, color='cyan', marker='o')
#     ax1.fill_between(
#         merged_data.index.values, 
#         merged_data.mean(axis=1) - merged_data.std(axis=1),
#         merged_data.mean(axis=1) + merged_data.std(axis=1),
#         color='cyan'
#     )
    
# nr_merged_stations = 0 if merged_data is None else len(merged_data.columns)
# print("%d stations were merged." % nr_merged_stations)
        
# Plot element observations
ax1.plot(wmo_obs_el['valid_datetime'].values, wmo_obs_el['value'].values, color='red', marker='o', markevery=6, label='WMO T')

# Plot radiation observations
if show_radiation:
    wmo_obs_qg = wmo_obs[wmo_obs.element_name=='qg']
    ax12 = ax1.twinx()
    ax12.plot(wmo_obs_qg['valid_datetime'].values, wmo_obs_qg['value'].values, color='green', marker='o', markevery=6, label='WMO QG')
    ax12.set_ylabel("Incoming radiation")
    
if show_grass:
    wmo_obs_qg = wmo_obs[wmo_obs.element_name=='tgn']
    ax1.plot(wmo_obs_qg['valid_datetime'].values, wmo_obs_qg['value'].values, color='blue', marker='o', markevery=6, label='WMO TGN')

if show_cloudiness:
    wmo_obs_n = wmo_obs[wmo_obs.element_name=='n']
    ax1.plot(wmo_obs_n['valid_datetime'].values, wmo_obs_n['value'].values, color='blue', marker='o', markevery=6, label='WMO N')

    
# == Second subplot ==
# # Take difference over rows
# merged_data_diff = merged_data.diff(axis=0)
# # Plot station diffs
# if show_individual:
#     for name in merged_data:
#         ax2.plot(merged_data_diff.index.values, merged_data_diff[name].values, marker='x', label=name)
# # Plot WMO diff
# ax2.plot(wmo_obs_el['valid_datetime'].values, wmo_obs_el['value'].diff().values, color='red', marker='o', markevery=6, label='WMO T')

# Plot cleanup
fig.autofmt_xdate()
plt.xticks(wmo_obs_el['valid_datetime'].values[0::144])  # Make ticks daily
plt.xlim([min(wmo_obs_el['valid_datetime'].values), max(wmo_obs_el['valid_datetime'].values)])
ax1.set_title("%s timeseries" % element)
ax2.set_title("%s diff timeseries" % element)
ax2.set_ylabel("diff per hour")
ax1.grid('on')
ax2.grid('on')
# ax1.legend(numpoints=1)
# ax2.legend(numpoints=1)
plt.show()